# Gettings started with Amazon `Bedrock`

![](images/bedrock_models.png)

### Create a `bedrock` client

In [1]:
import json
import boto3
import base64



#Create the connection to Bedrock
bedrock = boto3.client(
    service_name='bedrock',
    region_name='us-west-2', 
)

bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-west-2', 
    
)

### List of all available models

In [2]:
# List all foundation models
all_llms = [ model['modelId'] for model in bedrock.list_foundation_models()['modelSummaries']]
all_llms

['amazon.titan-tg1-large',
 'amazon.titan-embed-g1-text-02',
 'amazon.titan-text-lite-v1:0:4k',
 'amazon.titan-text-lite-v1',
 'amazon.titan-text-express-v1:0:8k',
 'amazon.titan-text-express-v1',
 'amazon.titan-embed-text-v1:2:8k',
 'amazon.titan-embed-text-v1',
 'amazon.titan-embed-image-v1:0',
 'amazon.titan-embed-image-v1',
 'amazon.titan-image-generator-v1:0',
 'amazon.titan-image-generator-v1',
 'stability.stable-diffusion-xl',
 'stability.stable-diffusion-xl-v0',
 'stability.stable-diffusion-xl-v1:0',
 'stability.stable-diffusion-xl-v1',
 'ai21.j2-grande-instruct',
 'ai21.j2-jumbo-instruct',
 'ai21.j2-mid',
 'ai21.j2-mid-v1',
 'ai21.j2-ultra',
 'ai21.j2-ultra-v1',
 'anthropic.claude-instant-v1:2:100k',
 'anthropic.claude-instant-v1',
 'anthropic.claude-v2:0:18k',
 'anthropic.claude-v2:0:100k',
 'anthropic.claude-v2:1:18k',
 'anthropic.claude-v2:1:200k',
 'anthropic.claude-v2:1',
 'anthropic.claude-v2',
 'anthropic.claude-3-sonnet-20240229-v1:0',
 'cohere.command-text-v14:7:4k',


### Define prompt and model parameters 

### With `Amazon Titan`

In [ ]:
prompt_data = """Write me a poem about apples"""

text_gen_config = {
                    "maxTokenCount": 512,
                    "stopSequences": [], 
                    "temperature": 0,
                    "topP": 0.9
                }

body = json.dumps({
                        "inputText": prompt_data,
                        "textGenerationConfig": text_gen_config  
                    })

model_id = 'amazon.titan-tg1-large'
accept = 'application/json' 
content_type = 'application/json'

#### Invoke model 


In [ ]:
response = bedrock_runtime.invoke_model(
                                            body=body, 
                                            modelId=model_id, 
                                            accept=accept, 
                                            contentType=content_type
                                        )

#### Print response

In [ ]:
response_body = json.loads(response['body'].read())
print(response_body['results'][0]['outputText'])

### With `Anthropic (Claude v3 (Sonnet))`

In [3]:
prompt_data = """What is L in LLM means"""

body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1024,
            "messages": [
                 {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt_data
                        }
                    ]
                }
            ],
        }

body = json.dumps(body) # Encode body as JSON string

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = 'application/json'
contentType = 'application/json'

#### Invoke model 


In [4]:
response = bedrock_runtime.invoke_model(body=body,
                                        modelId=model_id, 
                                        accept=accept, 
                                        contentType=contentType)

#### Print response

In [5]:
response_body = json.loads(response.get("body").read())
for output in response_body.get("content", []):
    print(output["text"])

In the context of LLM, the 'L' stands for 'Large' Language Model.

An LLM (Large Language Model) refers to a type of advanced language model that has been trained on a massive amount of text data from the internet and other sources. These models are called "large" because they typically have billions or even trillions of parameters, making them significantly larger and more complex than earlier language models.

Some key characteristics of LLMs:

1) Massive training data: LLMs are trained on vast corpora of text from diverse sources like websites, books, articles etc. This allows them to learn patterns and relationships across a wide range of topics.

2) Self-attention mechanism: They employ transformer architectures with self-attention mechanisms that can capture long-range dependencies better than RNNs.

3) Multi-task capability: With sufficient training data, LLMs can perform well on a variety of natural language tasks like text generation, question answering, summarization etc.

4)

### With `Mistral (mixtral-8x7b)`

In [ ]:
prompt_data = """<s>[INST]Craft a Python function to convert Celsius to Fahrenheit. If water boils at 100°C, what's that in Fahrenheit?[/INST]"""

body = json.dumps({ 
    'prompt': prompt_data,
    'max_tokens': 200,
    'top_p': 0.9,
    'temperature': 0.2,
})


modelId = 'mistral.mixtral-8x7b-instruct-v0:1'
accept = 'application/json'
contentType = 'application/json'

#### Invoke model 


In [ ]:
response = bedrock_runtime.invoke_model(body=body.encode('utf-8'), # Encode to bytes
                                        modelId=modelId, 
                                        accept=accept, 
                                        contentType=contentType)

#### Print response

In [ ]:
response_body = json.loads(response.get('body').read().decode('utf-8'))
print(response_body.get('outputs')[0].get('text'))

### Syntax Multi-Modal models from `Anthropic Claude v3 Models`

In [ ]:
with open("images/cat.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read())
    base64_string = encoded_string.decode('utf-8')

payload = {
    "modelId": "anthropic.claude-3-sonnet-20240229-v1:0",
    "contentType": "application/json",
    "accept": "application/json",
    "body": {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": base64_string
                        }
                    },
                    {
                        "type": "text",
                        "text": "Write me a detailed description of this photo."
                    }
                ]
            }
        ]
    }
}

# Convert the payload to bytes
body_bytes = json.dumps(payload['body']).encode('utf-8')

# Invoke the model
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = 'application/json'
contentType = 'application/json'

response = bedrock_runtime.invoke_model(body=body_bytes,
                                        modelId=model_id, 
                                        accept=accept, 
                                        contentType=contentType)


In [ ]:
response_body = json.loads(response.get('body').read().decode('utf-8'))
print(response_body.get('content')[0].get('text'))